# GSP Final Project
In this notebook, we will demonstrate estimation of graph signals using a GSP-based technique vs.  a deep-learning-based technique. 

In [2]:
import util
import torch
import torch.nn as nn
import numpy as np
import scipy.sparse as ssparse
import scipy.io as sio
import torch_geometric as tg
import matplotlib.pyplot as plt
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader

In [2]:
# Enable CUDA
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Mon Apr 22 17:41:56 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 552.22                 Driver Version: 552.22         CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                     TCC/WDDM  | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 4060 ...  WDDM  |   00000000:01:00.0  On |                  N/A |
| N/A   38C    P8              3W /   55W |     715MiB /   8188MiB |      3%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

## Task

$$\mathbf{x} \longrightarrow  \boxed{\text{Physical Model}} \longrightarrow \mathbf{y}  \longrightarrow \boxed{\text{Estimator}}  \longrightarrow \hat{\mathbf{x}}$$ 

### Physical Model
$$ \mathbf{y} = \mathbf{g}(\mathbf{x};\mathbf{L}) + \mathbf{w} $$

- $\mathbf{x} \sim p(\mathbf{x})$.
- $\mathbf{g}$ - a non-linear measurement function.
- $\mathbf{L}$ - the Laplacian matrix of the graph.
 
### Estimator
- $\mathbf{y}$ - input of the estimator
- $\mathbf{x}$ - ground-truth corresponding label
- The estimator is given a dataset pairs of $\{\mathbf{x_t}, \mathbf{y_t} \}_t$ for training.

**Goal**: The estimator should recover $\mathbf{x}$ out of $\mathbf{y}$ with minimum MSE.
#### GSP-based

Use a GSP-based technique for estimation.

#### GNN-based

Use a GNN deep-learning architecture for estimation.

**NOTE:**  $\mathbf{y}$ is the input to the DP model and $\mathbf{x}$ is the output (i.e., the label) not vise versa!

## Part 1: Physical Model

In [3]:
def laplacian_evd(Y):
    L = - np.imag(Y)
    Lambda, V = np.linalg.eig(L)
    Lambda = np.real(Lambda)
    sorted_indices = np.argsort(Lambda)
    Lambda = Lambda[sorted_indices]
    Lambda = np.diag(Lambda)
    V = V[:, sorted_indices]
    return L, Lambda, V
    
def g_xL(Y, x):
    v = np.exp(1j * x)
    g_x = np.real(v * np.conj(Y @ v))
    return g_x
    
def generate_data(nt, Y, Lambda, V, beta=3, c_ww=0.05):
    N = Y.shape[0]
    xt = (V[:, 1:] @ np.random.multivariate_normal(np.zeros(N - 1), beta * np.diag(1 / np.diag(Lambda)[1:]), nt).T).T
    if nt == 1:
        xt = xt[0, :]
    
    yt = np.zeros(xt.shape)
    for t in range(0, nt):
        yt[t] = g_xL(Y, xt[t])
    yt += np.sqrt(c_ww) * np.random.randn(yt.shape[0], yt.shape[1])
    
    return xt, yt

## Part 2: GSP-LMMSE Estimator

The GSP-LMMSE estimator is defined as an estimator which minimize the MSE among all estimators in the form of a graph filter:
$$
\{\bf{h}, \bf{b} \} = \text{argmin}~ \mathbb{E} [(\bf{x} - \hat{\bf{x}}(\bf{y}))^2]
$$

where $\hat{\bf{x}}(\bf{y}) =  \bf{V} \text{diag} (\bf{h}) \bf{V}^T \bf{y}+ \bf{b}$.

A closed form expression would be:
$$ \hat{\bf{x}}(\bf{y}) =  \bf{V} \text{diag} (\bf{d}_{\bf{xy}}\oslash \bf{d}_{\bf{yy}}) \bf{V}^T \bf{y} + \bar{\bf{x}}$$

where $\bf{d}_{\bf{xy}} := \text{diag}(\text{cov}(\bf{V}^T \bf{x}, \bf{V}^T \bf{y}))$, $\bf{d}_{\bf{yy}} := \text{diag}(\text{var}(\bf{V}^T \bf{y}))$ and $\bar{\bf{x}} :=\mathbb{E}\bf{x}$

In [4]:
def train_gsp_lmmse_estimator(xt, yt, V):
    xt = xt.T     # column vectors representation
    yt = yt.T     # column vectors representation

    xt_mean = np.mean(xt, axis=1)[:, np.newaxis]
    yt_mean = np.mean(yt, axis=1)[:, np.newaxis]

    d_xy = np.mean( ( V.T @ (xt - xt_mean) ) * ( V.T @ (yt - yt_mean) ) , axis=1)

    d_yy = np.mean( ( V.T @ (yt - yt_mean) ) ** 2 , axis=1)
    
    h = d_xy / d_yy
    
    return h, xt_mean

## Part 3: GNN and Deep Learning based Estimation

### Define Model Architecture

In [15]:
class GNN(torch.nn.Module):
    def __init__(self, num_features, K = 10 ):  # K is the order of the Chebyshev polynomial
        super(GNN, self).__init__()
        
        self.conv1 = tg.nn.ChebConv(num_features, 118, K=K, normalization=None)
        # self.relu1 = nn.ReLU()

    def forward(self, x, edge_index, edge_weight):
        print(f"x: {x.shape}, edge_index: {edge_index.shape}, edge_weight: {edge_weight.shape}")
        x = self.conv1(x, edge_index)
        # x = self.relu1(x)
        return x


(### Define training function

In order to run on GPU via CUDA set device accordingly.

In [21]:
def train_model(model, yt_train, xt_train, yt_valid, xt_valid, edge_index, edge_weight, batch_size=117, valid_batch_size = 50, epochs=40, lr=0.001, weight_decay=1e-4, path=None, device='cpu'):
    
    # train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
    # val_loader = DataLoader(valid_data, batch_size=valid_batch_size, shuffle=False)

    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
    
    train_mse, valid_mse = [], []
    
    for epoch in range(epochs):
        idx = np.random.permutation(len(yt_train))
        yt_train = yt_train[idx]
        xt_train = xt_train[idx]
        
        # Training
        model.train() 
        train_loss = 0
        iters = 0
        for i in range(0, len(yt_train), batch_size):
            if i + batch_size > len(yt_train):
                break
            
            yt = yt_train[i: i + batch_size]
            yt = torch.tensor(yt, dtype=torch.float32)
            
            xt = xt_train[i: i + batch_size]
            xt = torch.tensor(xt, dtype=torch.float32)
            
            data = Data(x=yt, edge_index=edge_index, y=xt)
            data = data.to(device)
            edge_index = edge_index.to(device)
            edge_weight = edge_weight.to(device)

            optimizer.zero_grad()
            output = model(data.x, edge_index, edge_weight)
            loss = criterion(output,data.y)
            loss.backward()
            optimizer.step()
            train_loss += float(loss.item())
            iters += 1
        train_mse.append(train_loss / iters)
        # 
        # # Validation
        # model.eval()
        # valid_loss = 0
        # iters = 0
        # with torch.no_grad():
        #     for i in range(0, len(yt_valid), valid_batch_size):
        #         yt = yt_valid[i: i + valid_batch_size]
        #         yt = torch.tensor(yt, dtype=torch.float32, device=device)
        #     
        #         xt = xt_valid[i: i + valid_batch_size]
        #         xt = torch.tensor(xt, dtype=torch.float32, device=device)
        #         
        #         xt_hat = model(yt, edge_index, edge_weight)
        #         loss = criterion(xt,xt_hat)
        #         valid_loss += float(loss.item())
        #         iters += 1
        #     valid_mse.append(valid_loss / iters)
        # 
        # if valid_mse[-1] == max(valid_mse) and path is not None:
        #     print("Current State Saved")
        #     torch.save(model.state_dict(), path)
        # 
        # print(f"Epoch: {epoch}, Train MSE {train_mse[-1]}, Validation MSE {valid_mse[-1]}")
    
    valid_mse = 0
    return train_mse, valid_mse
        
        
def plot_learning_curve(train_mse, valid_mse):
    plt.figure(figsize=(10, 5))
    plt.plot(train_mse, label='Training MSE', color='blue')
    plt.plot(valid_mse, label='Validation MSE', color='black')
    plt.title('Training and Validation Losses')
    plt.xlabel('Epochs')
    plt.ylabel('MSE')
    plt.legend()
    plt.show()

## Part 4: Train
### Prepare Data

In [7]:
filename = 'grid_data_ieee118cdf.mat'
grid_data = sio.loadmat(filename)
Y = grid_data['Y']
N = Y.shape[0]
nt = 10000

L, Lambda, V = laplacian_evd(Y)
xt_total, yt_total = generate_data(nt, Y, Lambda, V, )

### Train GSP-LMMSE Estimator

In [7]:
h_gsp, b_gsp = train_gsp_lmmse_estimator(xt_total, yt_total, V)

### Train GNN Model

In [22]:
path = "./best_GNN_model.pk"
GNN_model = GNN(num_features=N)

L_sparse = ssparse.csr_matrix(np.diag(np.diag(L)) -L)
edge_index, edge_weight = tg.utils.from_scipy_sparse_matrix(L_sparse)

device = 'cpu'

# NOTE: yt is the input for the model, and xt is the label, in Data object x is the input and y is the label!
yt_train, xt_train = yt_total[:int(len(yt_total) * 0.8)], xt_total[:int(len(xt_total) * 0.8)]
yt_valid, xt_valid = yt_total[int(len(yt_total) * 0.8):], xt_total[int(len(xt_total) * 0.8):]

GNN_model.to(device)

train_mse, valid_mse = train_model(GNN_model, yt_total, xt_total, yt_valid, xt_valid, edge_index, edge_weight, path=path)
# plot_learning_curve(train_mse, valid_mse)

x: torch.Size([117, 118]), edge_index: torch.Size([2, 358]), edge_weight: torch.Size([358])


RuntimeError: index 117 is out of bounds for dimension 0 with size 117

## Part 4: Comparison

In [25]:
import torch
import torch_geometric
from torch_geometric.nn import ChebConv
from torch_geometric.data import Data

# Suppose each node initially has 1 feature (could be a simple constant value)
num_nodes = 118
num_features = 1  # Number of input features
out_features = 1  # Number of output features you want after convolution
K = 2  # Chebyshev polynomial degree

# Create a simple feature matrix with constant features for all nodes
x = torch.ones((num_nodes, num_features))

# Define your edge indices (example only, replace with your graph's edges)
# edge_index must be a 2 x num_edges tensor indicating node connections
edge_index = torch.tensor([[0, 1, 1, 2], [1, 0, 2, 1]], dtype=torch.long)

# Optionally, you can specify edge weights if your graph is weighted
# edge_weight = torch.tensor([...], dtype=torch.float)

# Create a PyG graph data object
data = Data(x=x, edge_index=edge_index)

# Initialize the Chebyshev convolution layer
conv = ChebConv(in_channels=num_features, out_channels=out_features, K=K)

# Apply the convolution
x = conv(x, edge_index)  # if you have edge weights, add: , edge_weight=edge_weight

# x now contains the new features of your nodes after the convolution
print(x.shape)


torch.Size([118, 1])


In [ ]:
import matplotlib.pyplot as plt


import torch
import numpy as np
import scipy.sparse as sp
from torch_geometric.utils import from_scipy_sparse_matrix
import torch.nn.functional as F

from torch_geometric.data import Data
# Convert the NumPy array to a scipy sparse matrix (for example, using CSR format)
Y_sparse = sp.csr_matrix(grid_init.Y)

# Now, use from_scipy_sparse_matrix to convert Y_sparse to edge_index
edge_index, _ = from_scipy_sparse_matrix(Y_sparse)



import torch
import numpy as np
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv, ChebConv
import torch.nn.functional as F

x = torch.tensor(x_p_test, dtype=torch.float)
y = torch.tensor(y_p_test, dtype=torch.float)

# Instantiate the MSELoss object
mse_loss_fn = torch.nn.MSELoss()

# Calculate the MSE loss between x and y
loss = mse_loss_fn(x, y)

# Print the loss
print(loss.item())
num_nodes = x.size(0)


# Randomly select 80% of the indices for training, the rest for testing
indices = np.random.permutation(num_nodes)




# Create a single Data object
data = Data(x=x, edge_index=edge_index, y=y,)

edge_index, edge_weight = from_scipy_sparse_matrix(Y_sparse)

from torch_geometric.nn import ChebConv

class GNN(torch.nn.Module):
    def __init__(self, num_features, num_classes=118, K=3):  # K is the order of the Chebyshev polynomial
        super(GNN, self).__init__()
        self.conv1 = ChebConv(num_features, 118, K=K)
        # You can add more layers if needed

    def forward(self, x, edge_index, edge_weight=None):
        # If your graph is weighted, pass edge_weight to ChebConv
        x = self.conv1(x, edge_index, edge_weight)
        # Apply further processing if necessary
        return x

model = GNN(num_features=x.size(1))  # Adjust num_features based on your x tensor
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.MSELoss()

def train():
    model.train()
    optimizer.zero_grad()
    out = model(data.y[:8000], data.edge_index)
    squared_diff = (out- data.x[:8000])** 2


    # Step 2: Average over nodes
    mean_squared_diff_per_example = torch.mean(squared_diff, dim=1)

    # Step 3: Average over the dataset
    loss = torch.mean(mean_squared_diff_per_example)
    loss.backward()
    optimizer.step()
    return loss.item()
def test():
    model.eval()
    with torch.no_grad():
        out = model(data.y[8000:], data.edge_index)
        squared_diff = (out- data.x[8000:]) ** 2

        # Step 2: Average over nodes
        mean_squared_diff_per_example = torch.mean(squared_diff, dim=1)

        # Step 3: Average over the dataset
        loss = torch.mean(mean_squared_diff_per_example)
    return loss.item()

for epoch in range(100):
    train_loss = train()
    test_loss = test()
    if epoch % 100==0:
        print(f'Epoch: {epoch+1}, Train Loss: {train_loss}, Test Loss: {test_loss}')

c_ww_values = np.linspace(0.01, 1, 10)
print(c_ww_values)

train_mse = []
test_mse = []
start_time_ml = time.time()

for i in range(10):
    # Generate data for the current c_ww
    x, y = util.generate_data_grid(P, grid_init, c_ww=c_ww_values[i])
    x = torch.tensor(x, dtype=torch.float)
    y = torch.tensor(y, dtype=torch.float)
    data = Data(x=x, edge_index=edge_index, y=y)

    # Initialize the model
    model = GNN(num_features=x.size(1))
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    criterion = torch.nn.MSELoss()


    # Your training and testing functions remain the same


    # Train and test the model
    for epoch in range(150):
        train_loss = train()

        test_loss = test()
        if epoch % 10 == 0:
            print(f'Epoch: {epoch + 1}, Train Loss: {train_loss}, Test Loss: {test_loss}')
    # Store the final MSE
    train_mse.append(train_loss)
    test_mse.append(test_loss)

end_time_ml = time.time()

duration_ml = end_time_ml - start_time_ml
print(f"ML Model Training and Testing Time: {duration_ml} seconds")

# Plotting the MSE vs. c_ww values
plt.figure(figsize=(10, 6))
plt.plot(1/c_ww_values, train_mse, label='Train MSE', marker='o', linestyle='-', color='b')
plt.plot(1/c_ww_values, test_mse, label='Test MSE', marker='o', linestyle='-', color='r')
plt.xscale('log')  # Set x-axis to log scale
plt.xlabel('1/c_ww (Inverse of Noise Variance)')
plt.ylabel('MSE')
plt.title('MSE vs. 1/c_ww on Log Scale')
plt.legend()
plt.grid(True, which="both", ls="--")
plt.show()

146.92237854003906
Epoch: 1, Train Loss: 305.4447021484375, Test Loss: 226.54116821289062
[0.01 0.12 0.23 0.34 0.45 0.56 0.67 0.78 0.89 1.  ]
Epoch: 1, Train Loss: 302.4858703613281, Test Loss: 220.28587341308594
Epoch: 11, Train Loss: 25.08798599243164, Test Loss: 29.456289291381836
Epoch: 21, Train Loss: 13.775940895080566, Test Loss: 18.41518211364746
Epoch: 31, Train Loss: 4.719074249267578, Test Loss: 10.501501083374023
Epoch: 41, Train Loss: 2.1199913024902344, Test Loss: 8.29074764251709
Epoch: 51, Train Loss: 0.962665855884552, Test Loss: 7.325508117675781
Epoch: 61, Train Loss: 0.5312516689300537, Test Loss: 7.0464982986450195
Epoch: 71, Train Loss: 0.3373253643512726, Test Loss: 6.977317810058594
Epoch: 81, Train Loss: 0.24654310941696167, Test Loss: 6.992523670196533
Epoch: 91, Train Loss: 0.198048397898674, Test Loss: 7.031910419464111
Epoch: 101, Train Loss: 0.16871300339698792, Test Loss: 7.077186584472656
Epoch: 111, Train Loss: 0.1493619978427887, Test Loss: 7.123420715